# Модуль 5

### Загрузка необходимых билиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import mean_squared_error, r2_score

import gc
import joblib
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Загрузка данных

In [2]:
X = pd.read_parquet('/home/c13/Загрузки/BIG DATA/data_X.parquet')
y = pd.read_parquet('/home/c13/Загрузки/BIG DATA/data_y.parquet')

In [3]:
# Для теста модели возмем 10000 записей
X = X.iloc[:5000]
y = y.iloc[:5000]

### Задание 1 Разработка модели машинного обучения

#### Обучающая выборка данных разделена на обучающую и тестовую выборку

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Нормализация данных. Набор данных имеет различные числовые значения, положительная практика нормальзовать такие данные.

In [6]:
scaler_X = MinMaxScaler()
X_train_normalized = scaler_X.fit_transform(X_train)
X_test_normalized = scaler_X.transform(X_test)  # Применяем тот же трансформер к тестовым данным

# Нормализация целевой переменной
scaler_y = MinMaxScaler()
y_train_normalized = scaler_y.fit_transform(y_train)
y_test_normalized = scaler_y.transform(y_test) 

In [5]:
scaler_X = StandardScaler()
X_train_normalized = scaler_X.fit_transform(X_train)
X_test_normalized = scaler_X.transform(X_test)  # Применяем тот же трансформер к тестовым данным

# Нормализация целевой переменной
scaler_y = StandardScaler()
y_train_normalized = scaler_y.fit_transform(y_train)
y_test_normalized = scaler_y.transform(y_test) 

#### Random Forest без нормализации

In [6]:
# 1. MultiOutputRegressor с Random Forest
start_time = time.time()
rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


end_time = time.time()
execution_time = end_time - start_time
print(f"Время обучения RandomForestRegressor : {execution_time:.6f} секунд")


# Оценка модели Random Forest
print("Random Forest:")
print("RMSE (t0):", np.sqrt(mean_squared_error(y_test['t0'], y_pred_rf[:, 0])))
print("RMSE (t1):", np.sqrt(mean_squared_error(y_test['t1'], y_pred_rf[:, 1])))
print("R2 (t0):", r2_score(y_test['t0'], y_pred_rf[:, 0]))
print("R2 (t1):", r2_score(y_test['t1'], y_pred_rf[:, 1]))

Время обучения RandomForestRegressor : 9.098526 секунд
Random Forest:
RMSE (t0): 1214.9995544663634
RMSE (t1): 1287.664625342543
R2 (t0): -0.025642425913672717
R2 (t1): -0.01911569076379993


#### Random Forest с нормализацией

In [7]:
# 1. MultiOutputRegressor с Random Forest
start_time = time.time()
rf_model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
rf_model.fit(X_train_normalized, y_train_normalized)
y_pred_rf = rf_model.predict(X_test_normalized)


end_time = time.time()
execution_time = end_time - start_time
print(f"Время обучения RandomForestRegressor : {execution_time:.6f} секунд")


# Оценка модели Random Forest
print("Random Forest:")
print("RMSE (t0):", np.sqrt(mean_squared_error(y_test_normalized[:, 0], y_pred_rf[:, 0])))
print("RMSE (t1):", np.sqrt(mean_squared_error(y_test_normalized[:, 1], y_pred_rf[:, 1])))
print("R2 (t0):", r2_score(y_test_normalized[:, 0], y_pred_rf[:, 0]))
print("R2 (t1):", r2_score(y_test_normalized[:, 1], y_pred_rf[:, 1]))

Время обучения RandomForestRegressor : 9.020569 секунд
Random Forest:
RMSE (t0): 1.0486213522490453
RMSE (t1): 1.0229441853207752
R2 (t0): -0.025833827857751457
R2 (t1): -0.020003797738827434


#### Gradient Boosting  без нормализации

In [8]:
# 2. MultiOutputRegressor с Gradient Boosting
start_time = time.time()
gb_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

end_time = time.time()
execution_time = end_time - start_time
print(f"Время обучения GradientBoostingRegressor : {execution_time:.6f} секунд")


# Оценка модели Gradient Boosting
print("\nGradient Boosting:")
print("RMSE (t0):", np.sqrt(mean_squared_error(y_test['t0'], y_pred_gb[:, 0])))
print("RMSE (t1):", np.sqrt(mean_squared_error(y_test['t1'], y_pred_gb[:, 1])))
print("R2 (t0):", r2_score(y_test['t0'], y_pred_gb[:, 0]))
print("R2 (t1):", r2_score(y_test['t1'], y_pred_gb[:, 1]))

Время обучения GradientBoostingRegressor : 2.918318 секунд

Gradient Boosting:
RMSE (t0): 1208.8768582919683
RMSE (t1): 1288.761906611424
R2 (t0): -0.015331517686623553
R2 (t1): -0.020853306283420414


#### Gradient Boosting с нормализацией

In [9]:
# 2. MultiOutputRegressor с Gradient Boosting
start_time = time.time()
gb_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
gb_model.fit(X_train_normalized, y_train_normalized)
y_pred_gb = gb_model.predict(X_test_normalized)

end_time = time.time()
execution_time = end_time - start_time
print(f"Время обучения GradientBoostingRegressor : {execution_time:.6f} секунд")


# Оценка модели Gradient Boosting
print("\nGradient Boosting:")
print("RMSE (t0):", np.sqrt(mean_squared_error(y_test_normalized[:, 0], y_pred_gb[:, 0])))
print("RMSE (t1):", np.sqrt(mean_squared_error(y_test_normalized[:, 1], y_pred_gb[:, 1])))
print("R2 (t0):", r2_score(y_test_normalized[:, 0], y_pred_gb[:, 0]))
print("R2 (t1):", r2_score(y_test_normalized[:, 1], y_pred_gb[:, 1]))

Время обучения GradientBoostingRegressor : 2.931617 секунд

Gradient Boosting:
RMSE (t0): 1.043169905911594
RMSE (t1): 1.023489872214675
R2 (t0): -0.01519558971572943
R2 (t1): -0.021092324701041054


#### LinearRegression без нормализации

In [10]:
start_time = time.time()
lr_model = MultiOutputRegressor(LinearRegression())
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

end_time = time.time()
execution_time = end_time - start_time
print(f"Время обучения LinearRegression : {execution_time:.6f} секунд")


# Оценка модели LinearRegression
print("\nLinearRegression:")
print("RMSE (t0):", np.sqrt(mean_squared_error(y_test['t0'], y_pred_gb[:, 0])))
print("RMSE (t1):", np.sqrt(mean_squared_error(y_test['t1'], y_pred_gb[:, 1])))
print("R2 (t0):", r2_score(y_test['t0'], y_pred_gb[:, 0]))
print("R2 (t1):", r2_score(y_test['t1'], y_pred_gb[:, 1]))

Время обучения LinearRegression : 0.032915 секунд

LinearRegression:
RMSE (t0): 2968.9402256253197
RMSE (t1): 3309.758690736903
R2 (t0): -5.12416168035081
R2 (t1): -5.73303110429928


### Обоснование модели
##### LinearRegression показывает неудовлетворительные результаты, Gradient Boosting и Random Forest результаты примерно одинаковые, но Gradient Boosting обучается быстрее. Нормализация не дает улучшения. За основную модель возмем Gradient Boosting без нормализации данных.

In [12]:
def train_model(train_file: str, target_file: str, val_file: str, val_target_file: str):
    # Загрузка данных
    X = pd.read_parquet(train_file)
    X = X.reset_index(drop=True)
    y = pd.read_parquet(target_file)
    y = y.reset_index(drop=True)

    val = pd.read_parquet(train_file)
    val = val.reset_index(drop=True)
    y_val = pd.read_parquet(target_file)
    y_val = y_val.reset_index(drop=True)

    # Обучение модели
    model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
    model.fit(X_train, y_train)

    # Предсказание на валидационной выборке
    y_pred = model.predict(val)

    # Вычисление метрик
    metrics = {
        'R2': r2_score(y_val, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_val, y_pred))
    }

    # Сохранение модели
    joblib.dump(model, 'trained_model.pkl')

    return model, metrics


In [13]:
train_file_path = '/home/c13/Загрузки/BIG DATA/data_X.parquet'
target_file_path = '/home/c13/Загрузки/BIG DATA/data_y.parquet'
val_file_path = '/home/c13/Загрузки/BIG DATA/data_X_val.parquet'
val_target_file_path = '/home/c13/Загрузки/BIG DATA/data_y_val.parquet'

In [ ]:
# Загруженны данные имееют слишком большой объем, на данной машине нет смысла запускать обучение на всей выборке. Но я поробую!
model, metrics = train_model(train_file_path, target_file_path, val_file_path, val_target_file_path)

print("Метрики модели на валидационной выборке:")
print(metrics)

# Отчет
#### Загруженны parquet данные с помощью pandas
#### Обучены три модели, сравнены по метрикам R2 и RMSE. А также по скорости работы.
#### Выбрана лучшая модель.
#### Реализован пайплайн для обучения модели